In [1]:
import pandas as pd
import pickle

In [2]:
palabras_lemm = 'palabras.pickle'
with open(palabras_lemm, "rb") as f:
    df_lemmas = pickle.load(f)

In [3]:
colores_grupos = {
    'Grupo Parlamentario Ciudadanos':'#FF571F',
    'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común':'#693279',
    'Grupo Parlamentario Popular en el Congreso': '#184A90',
    'Grupo Parlamentario Socialista': '#DC0612',
    'Grupo Parlamentario VOX': '#63BE21'
}
color_otro = '#ACB0B2'

In [6]:
from bokeh.io import show, output_notebook, save
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool, ColumnDataSource
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4
from bokeh.plotting import from_networkx
import numpy as np
from bokeh.models import LabelSet
import networkx as nx

output_notebook()

colores_grupos = {
    'Grupo Parlamentario Ciudadanos':'#FF571F',
    'Grupo Parlamentario Confederal de Unidas Podemos-En Comú Podem-Galicia en Común':'#693279',
    'Grupo Parlamentario Popular en el Congreso': '#184A90',
    'Grupo Parlamentario Socialista': '#DC0612',
    'Grupo Parlamentario VOX': '#63BE21'
}
color_otro = '#ACB0B2'


G = nx.Graph()

archivo = 'PR_DISCURSO_2020_01_05'

df_proceso = pd.read_csv(f'resultados_analisis\{archivo}.csv')
palabras_frecuentes = df_proceso.groupby('palabra')['veces'].sum().reset_index()
palabras_frecuentes = palabras_frecuentes.sort_values(by='veces', ascending=False)
top_palabras = palabras_frecuentes.head(20)['palabra'].tolist()
df_proceso = df_proceso[df_proceso['palabra'].isin(top_palabras)]
       
        
diccionario_grupo_parlamentario = df_proceso.groupby(['grupo_parlamentario'])['veces'].sum().reset_index().set_index('grupo_parlamentario')['veces'].to_dict()


def normalize_size(value, max_value, min_value):
    rango_max = 50
    rango_min = 5
    # Calculamos la proporción que representa value dentro del rango max_value-min_value
    prop = (value - min_value) / (max_value - min_value)
    # Calculamos el valor correspondiente en el rango 
    new_value = prop * (rango_max - rango_min) + rango_min
    # Devolvemos el resultado
    return new_value

max_total = df_proceso['total'].max()
min_total = df_proceso['total'].min()
max_veces = df_proceso['veces'].max()
min_veces = df_proceso['veces'].min()

for index, row in df_proceso.iterrows():
    if row["grupo_parlamentario"] not in G.nodes:
        size = normalize_size(diccionario_grupo_parlamentario[row["grupo_parlamentario"]], max(diccionario_grupo_parlamentario.values()), min(diccionario_grupo_parlamentario.values()))
        G.add_node(row["grupo_parlamentario"], size=size, tipo="grupo_parlamentario", nombre=row["grupo_parlamentario"], color=colores_grupos.get(row["grupo_parlamentario"], color_otro))
#        G.add_node(row["grupo_parlamentario"], size=20, tipo="grupo_parlamentario", nombre=row["grupo_parlamentario"], color=colores_grupos.get(row["grupo_parlamentario"], color_otro))
    if row["palabra"] not in G.nodes:
        G.add_node(row["palabra"], size=normalize_size(row["total"], max_total, min_total), tipo="palabra", palabra=row["palabra"], total=row["total"], color='white')
    G.add_edge(row["grupo_parlamentario"], row["palabra"], weight=row["veces"], color=colores_grupos.get(row["grupo_parlamentario"], color_otro))   

plot = Plot(plot_width=800, plot_height=800,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1), title= archivo, background_fill_color='black')

graph_renderer = from_networkx(G, nx.spring_layout(G, k=2), scale=1, center=(0, 0))

# Agrega nodos y enlaces al Plot
graph_renderer.node_renderer.data_source.data['fill_color'] = [node[1]['color'] for node in G.nodes(data=True)]
graph_renderer.node_renderer.glyph = Circle(size='size', fill_color='fill_color')

def make_line_width(weight, max_value):
    rango_max = 10
    rango_min = 1
    # Calculamos la proporción que representa value dentro del rango max_value-min_value
    prop = (weight - 1) / (max_value - 1)
    # Calculamos el valor correspondiente en el rango 5-50
    new_value = prop * (rango_max - rango_min) + rango_min
    # Devolvemos el resultado
    return new_value

graph_renderer.edge_renderer.data_source.data['line_width'] = [make_line_width(edge[2]['weight'], max_veces) for edge in G.edges(data=True)]
graph_renderer.edge_renderer.glyph = MultiLine(line_color='color', line_alpha=0.7, line_width='line_width')

plot.renderers.append(graph_renderer)

hover = HoverTool(tooltips=[("Tipo", "@tipo"),
                            ("Palabra", "@palabra"),
                            ("Nombre", "@nombre")])
plot.add_tools(hover, TapTool(), BoxSelectTool())


#Add Labels
x, y = zip(*graph_renderer.layout_provider.graph_layout.values())
node_labels = list(G.nodes())
source = ColumnDataSource({'x': x, 'y': y, 'name': [node_labels[i] for i in range(len(x))]})
labels = LabelSet(x='x', y='y', text='name', source=source, background_fill_color='white', text_font_size='10px', background_fill_alpha=.7)
plot.renderers.append(labels)

show(plot)
print(df_proceso.groupby(['palabra'])['veces'].sum().reset_index().sort_values(by='veces', ascending=False, inplace=False).head(40))
#

Loading BokehJS ...

         palabra  veces
12      político     55
11       navarra     54
16    socialista     51
6     democracia     50
5       defender     49
9         hablar     48
2   constitución     45
19         vasco     41
7        derecha     35
1          bildu     33
18       sánchez     32
17      sociedad     31
3         crisis     29
8       elección     29
0           ayer     28
13       popular     28
15      proyecto     27
4         cámara     26
14  precisamente     25
10      libertad     25


In [26]:
archivo_pr = 'PR_DISCURSO_2021_02_24_GS'
pd.read_csv(f'resultados_analisis\{archivo_pr}.csv').rename(columns={'total': 'PageRank'}).head(10)

,grupo_parlamentario,veces,palabra,fecha,PageRank
0,Grupo Parlamentario Socialista,84,pandemia,2021-02-24,17.575118
1,Grupo Parlamentario Socialista,76,señoer,2021-02-24,15.872485
2,Grupo Parlamentario Socialista,67,económico,2021-02-24,12.907178
3,Grupo Parlamentario Socialista,60,euros,2021-02-24,12.850711
4,Grupo Parlamentario Socialista,70,autónomo,2021-02-24,12.559948
5,Grupo Parlamentario Socialista,58,comunidad,2021-02-24,12.064464
6,Grupo Parlamentario Socialista,58,sector,2021-02-24,12.042247
7,Grupo Parlamentario Socialista,51,democracia,2021-02-24,11.515794
8,Grupo Parlamentario Socialista,50,libertad,2021-02-24,11.003076
9,Grupo Parlamentario Socialista,51,recuperación,2021-02-24,10.788941


In [28]:
archivo_com = 'COM_DISCURSO_2021_02_24_GS'
com = pd.read_csv(f'resultados_analisis\{archivo_com}.csv')
comunidad =  int(com[com['palabra'] == 'democracia']['comunidad'])
com[com['comunidad'] == comunidad]

,palabra,polaridad,comunidad
135,abandonar,0.0,54
136,abascal,0.3,54
137,abocar,0.3,54
138,abordamos,0.3,54
139,abro,0.3,54
...,...,...,...
1071,xvi,0.3,54
1072,época,0.3,54
1073,índice,0.3,54
1074,íntegro,0.3,54
